# AI Chess player

In [21]:
import chess
import chess.svg
from time import sleep
import matplotlib.pyplot as plt
import numpy as np

In [22]:
BOARD_SCORES = {
    "PAWN": 1,
    "BISHOP": 4,
    "KING": 0,
    "QUEEN": 10,
    "KNIGHT": 5,
    "ROOK": 3
}

END_SCORES = {
    "WIN": 100,
    "LOSE": -100,
    "TIE": 0,
}

PIECES = {
    1: "PAWN",
    2: "KNIGHT",
    3: "BISHOP",
    4: "ROOK",
    5: "QUEEN",
    6: "KING"
}

In [23]:
opening_moves = [
    chess.Move.from_uci("e2e4"),
    chess.Move.from_uci("d2d4"),
    chess.Move.from_uci("g1f3"),
    chess.Move.from_uci("c2c4"),
]

In [24]:
def evaluate(board):
    score = 0
    for piece in board.piece_map().values():
        if piece.color:
            score += BOARD_SCORES[PIECES[piece.piece_type]]
        else:
            score -= BOARD_SCORES[PIECES[piece.piece_type]]

    if board.is_checkmate():
        if board.turn:
            score += END_SCORES["LOSE"]
        else:
            score += END_SCORES["WIN"]
    elif board.is_stalemate():
        score += END_SCORES["TIE"]
    return score

In [25]:
def minimax_alpha_beta(board, depth, alpha, beta, is_maximizing):
    if depth == 0 or board.is_game_over():
        return evaluate(board)
    
    if is_maximizing:
        max_eval = float('-inf')
        for move in board.legal_moves:
            board.push(move)
            eval_score = minimax_alpha_beta(board, depth - 1, alpha, beta, False)
            board.pop()
            if eval_score is not None:
                max_eval = max(max_eval, eval_score)
                alpha = max(alpha, eval_score)
                if beta <= alpha:
                    break
        return max_eval
    else:
        min_eval = float('inf')
        for move in board.legal_moves:
            board.push(move)
            eval_score = minimax_alpha_beta(board, depth - 1, alpha, beta, True)
            board.pop()
            if eval_score is not None:
                min_eval = min(min_eval, eval_score)
                beta = min(beta, eval_score)
                if beta <= alpha:
                    break
        return min_eval

In [26]:
def get_best_move(board, depth):
    best_eval = float('-inf')
    best_move = None
    alpha = float('-inf')
    beta = float('inf')
    for move in board.legal_moves:
        board.push(move)
        eval_score = minimax_alpha_beta(board, depth - 1, alpha, beta, False)
        board.pop()
        if eval_score is not None:
            if eval_score > best_eval:
                best_eval = eval_score
                best_move = move
    return best_move

def print_board(board):
    board_str = str(board)
    board_list = board_str.split("\n")
    plt.clf()
    
    plt.text(0.1, 1, '  |a b c d e f g h', fontsize=20, fontname="monospace")
    plt.text(0.1, 0.9, '8 |' + board_list[0], fontsize=20, fontname="monospace")
    plt.text(0.1, 0.8, '7 |' + board_list[1], fontsize=20, fontname="monospace")
    plt.text(0.1, 0.7, '6 |' + board_list[2], fontsize=20, fontname="monospace")
    plt.text(0.1, 0.6, '5 |' + board_list[3], fontsize=20, fontname="monospace")
    plt.text(0.1, 0.5, '4 |' + board_list[4], fontsize=20, fontname="monospace")
    plt.text(0.1, 0.4, '3 |' + board_list[5], fontsize=20, fontname="monospace")
    plt.text(0.1, 0.3, '2 |' + board_list[6], fontsize=20, fontname="monospace")
    plt.text(0.1, 0.2, '1 |' + board_list[7], fontsize=20, fontname="monospace")
    plt.axis("off")

    plt.savefig("board.png")
    plt.show()
    sleep(0.5)

def play_chess_gui():
    board = chess.Board()

    if board.turn == chess.WHITE:
        # get a random move from the opening moves
        move = opening_moves[np.random.randint(0, len(opening_moves))]
        board.push(move)

    print(board)

    while not board.is_game_over():
        if board.turn == chess.WHITE:
            depth = 5
            move = get_best_move(board, depth)
            print("Computer move: ", move)
            board.push(move)
        else:
            user_input = input("Your move: ")
            while chess.Move.from_uci(user_input) not in board.legal_moves:
                user_input = input("Invalid move. Your move: ")
            while board.piece_at(chess.Move.from_uci(user_input).from_square).color == chess.WHITE:
                user_input = input("Invalid move. Your move: ")
            
            move = chess.Move.from_uci(user_input)
            board.push(move)
        # print(board)
        print_board(board)
        
    print("Game over. Result: ", board.result())

In [ ]:
play_chess_gui()